In [4]:
def split_str(string, n):
    return [string[i:i+n] for i in range(0, len(string), n)]

In [5]:
def left_rotate(b, n):
    return ((b << n) | (b >> (32 - n))) & 0xffffffff

In [6]:
def extend_by_80(chunk):
    for i in range(16, 80):
        word1 = chunk[i - 3]
        word2 = chunk[i - 8]
        word3 = chunk[i - 14]
        word4 = chunk[i - 16]

        xor1 = bin(int(word1, 2) ^ int(word2, 2))
        xor2 = bin(int(xor1, 2) ^ int(word3, 2))
        xor3 = bin(int(xor2, 2) ^ int(word4, 2))

        new_word = bin(left_rotate(int(xor3, 2), 1))
        chunk.append(new_word)
    return chunk

In [7]:
def proceed_chunk(chunk, h):
    a, b, c, d, e = h
    h0, h1, h2, h3, h4 = h
    for i in range(80):
        f = None
        k = None
        if i < 20:
            f = d ^ (b & (c ^ d))
            k = 0x5A827999
        elif i < 40:
            f = b ^ c ^ d
            k = 0x6ED9EBA1
        elif i < 60:
            f = (b & c) | (b & d) | (c & d)
            k = 0x8F1BBCDC
        else:
            f = b ^ c ^ d
            k = 0xCA62C1D6

        word = chunk[i]
        temp_a = left_rotate(a, 5) + f
        temp_b = temp_a + e
        temp_c = temp_b + k
        
        temp = temp_c + int(word, 2)
        temp = temp & 0xffffffff

        e = d
        d = c
        c = left_rotate(b, 30)
        b = a
        a = temp

    h0 = hex((h0 + a) & 0xffffffff)[2:]
    h1 = hex((h1 + b) & 0xffffffff)[2:]
    h2 = hex((h2 + c) & 0xffffffff)[2:]
    h3 = hex((h3 + d) & 0xffffffff)[2:]
    h4 = hex((h4 + e) & 0xffffffff)[2:]

    return ''.join([h0, h1, h2, h3, h4])

In [77]:
text = "hello, world"
text = list(map(ord, list(text)))
text

[104, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100]

In [78]:
text = list(map(bin, text))
text = [str(x)[2:] for x in text]
text

['1101000',
 '1100101',
 '1101100',
 '1101100',
 '1101111',
 '101100',
 '100000',
 '1110111',
 '1101111',
 '1110010',
 '1101100',
 '1100100']

In [79]:
text = [x.zfill(8) for x in text]
text

['01101000',
 '01100101',
 '01101100',
 '01101100',
 '01101111',
 '00101100',
 '00100000',
 '01110111',
 '01101111',
 '01110010',
 '01101100',
 '01100100']

In [80]:
''.join(text)

'011010000110010101101100011011000110111100101100001000000111011101101111011100100110110001100100'

In [81]:
binary_length = bin(len(''.join(text)))[2:]
binary_length

'1100000'

In [82]:
binary_length = binary_length.zfill(64)  # 2 305 843 TB
binary_length

'0000000000000000000000000000000000000000000000000000000001100000'

In [83]:
text = "".join(text) + "1"
text

'0110100001100101011011000110110001101111001011000010000001110111011011110111001001101100011001001'

In [84]:
while len(text) % 512 != 448:
    text += "0"
text, len(text)

('0110100001100101011011000110110001101111001011000010000001110111011011110111001001101100011001001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 448)

In [85]:
text += binary_length
text

'01101000011001010110110001101100011011110010110000100000011101110110111101110010011011000110010010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001100000'

In [86]:
text_chunks = split_str(text, 512)
text_chunks

['01101000011001010110110001101100011011110010110000100000011101110110111101110010011011000110010010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001100000']

In [87]:
text_chunks = [split_str(s, 32) for s in text_chunks]
text_chunks

[['01101000011001010110110001101100',
  '01101111001011000010000001110111',
  '01101111011100100110110001100100',
  '10000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000000000000',
  '00000000000000000000000001100000']]

In [88]:
h = (0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476, 0xC3D2E1F0)

In [89]:
for i in range(len(text_chunks)):
    text_chunks[i] = extend_by_80(text_chunks[i])
    h = proceed_chunk(text_chunks[i], h)

h

'b7e23ec29af22b0b4e41da31e868d57226121c84'